In [2]:
path="/code_search_net/python/"
training_data=f"{path}/train"
testing_data=f"{path}/test"
validation_data=f"{path}/validation"

In [ ]:
chat = [
   {"role": "user", "content": "What's the difference between PCA vs UMAP vs t-SNE?"},
   # We need to add the Assistant role as well, otherwise mlx_lm will error on generation.
   {"role": "assistant", "content": "The "},
]

prompt = tokenizer.apply_chat_template(chat, add_generation_prompt=True, tokenize=False)


In [3]:
from mlx_lm import load, generate

model, tokenizer = load("./fused_model")
chat = [
 {"content": "You are a code autocomplete agent. Users will input partial code snippets, and you will autocomplete the code based on the provided snippet.", "role": "system"}, 
 {"content": """def factorial(n): \n if n < 0: \n return 'Factorial is not defined for negative numbers.' \n result = 1 \n for i in range(1,0):""", "role": "user"},
{"content": "", "role": "assistant"}]

prompt = tokenizer.apply_chat_template(chat, add_generation_prompt=True, tokenize=False)

response = generate(model, tokenizer, prompt=prompt, max_tokens=80)


In [4]:
response

'<code>result *= i \nreturn result</code><|end|><|assistant|>'

In [24]:
import json
import random

def transform_jsonl(input_file, output_file, max_records=1000):
    # Read all lines from the input file
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    # Shuffle the lines to select random records
    random.shuffle(lines)

    # Select up to the specified number of records
    selected_lines = lines[:max_records]

    with open(output_file, 'w') as outfile:
        for line in selected_lines:
            item = json.loads(line)
            code = item['code']

            code_tokens = code.split()
            if len(code_tokens) > 1:
                split_idx = random.randint(1, len(code_tokens) - 1)

                user_code = ' '.join(code_tokens[:split_idx])
                assistant_code = ' '.join(code_tokens[split_idx:])
            else:
                user_code = code
                assistant_code = ''

            output_item = {
                "messages": [
                    {
                        "content": "You are a code autocomplete agent. Users will input partial code snippets, and you will autocomplete the code based on the provided snippet.",
                        "role": "system"
                    },
                    {
                        "content": user_code,
                        "role": "user"
                    },
                    {
                        "content": "<code>"+assistant_code+"</code>",
                        "role": "assistant"
                    }
                ]
            }

            outfile.write(json.dumps(output_item) + '\n')

transform_jsonl('./code_search_net/python/train/python_train_6.jsonl', './my-code-data/valid.jsonl', max_records=1000)
